In [1]:
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import pytorch_lightning as pl

import sys
sys.path.insert(0,'..')

from starreco.model import *
#from starreco.data import *

In [2]:
user_x = torch.FloatTensor(3760,10).random_(0, 5)
user_train_ds = TensorDataset(user_x, user_x)
user_train_dl = DataLoader(user_train_ds, batch_size = 1024, num_workers = 8)

user_mda = mDA(10, 5)
pl.Trainer(gpus = 1, max_epochs = 100, progress_bar_refresh_rate = 50, logger = False)\
.fit(user_mda, user_train_dl)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
/home/kyleong/anaconda3/envs/starreco/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 55    
1 | decoder | Sequential | 60    
---------------------------------------
115       Trainable params
0         Non-trainable params
115       Total params
0.000     Total estimated model params size (MB)


Epoch 99: 100%|██████████| 4/4 [00:00<00:00, 17.68it/s, loss=1.7e+04, train_loss_step=1.56e+4, train_loss_epoch=1.65e+4] 


1

In [3]:
item_x = torch.FloatTensor(6040,7).random_(0, 5)
item_train_ds = TensorDataset(item_x, item_x)
item_train_dl = DataLoader(item_train_ds, batch_size = 1024, num_workers = 8)

item_mda = mDA(7, 5)
pl.Trainer(gpus = 1, max_epochs = 100, progress_bar_refresh_rate = 50, logger = False)\
.fit(item_mda, item_train_dl)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 40    
1 | decoder | Sequential | 42    
---------------------------------------
82        Trainable params
0         Non-trainable params
82        Total params
0.000     Total estimated model params size (MB)


Epoch 99: 100%|██████████| 6/6 [00:00<00:00, 23.91it/s, loss=1.43e+05, train_loss_step=1.46e+5, train_loss_epoch=1.42e+5]


1

In [4]:
mdacf = mDACF(user_mda, item_mda, 
              weight_decay = 0) # Weight decay > 0 will updates freezed weights

for name, param in user_mda.named_parameters():
    print(name, param)
print("---")
for name, param in item_mda.named_parameters():
    print(name, param)
print("---")
for name, param in mdacf.named_parameters():
    print(name, param)

encoder.0.weight Parameter containing:
tensor([[-1.6301,  1.2406,  0.2073,  2.1855,  1.1885],
        [-0.2433, -0.2427, -1.3765, -1.4095, -1.9335],
        [-2.6377, -2.4519,  1.1827, -0.9560, -2.3007],
        [-2.4313, -2.0646, -2.4477, -2.5715, -0.8255],
        [-0.0619, -0.3231, -0.4773, -0.4651, -1.5533],
        [-1.3718, -1.3856, -1.3968,  0.0081, -2.0328],
        [ 1.7224,  0.4979,  0.3116,  0.2665,  0.4633],
        [-0.9868,  0.7585, -0.5694,  1.6053,  1.5704],
        [-1.0549,  2.1950,  1.2452,  0.2813, -0.3438],
        [ 2.1827, -0.6145, -2.1656, -1.5246,  0.1930]], requires_grad=True)
encoder.0.bias Parameter containing:
tensor([0.7653, 0.7463, 0.7799, 0.7474, 0.7712], requires_grad=True)
decoder.0.weight Parameter containing:
tensor([[-1.5801,  0.0823, -2.2548, -1.8616,  0.2046, -0.7802,  1.7302, -0.9573,
         -1.0228,  2.1623],
        [ 1.2416,  0.2807, -1.9206, -1.3982,  0.1066, -0.6785,  0.5265,  0.7678,
          2.1975, -0.5494],
        [ 0.2086, -0.8817, 

In [5]:
X = torch.FloatTensor(1000000,17).random_(0, 5)
y = torch.FloatTensor(1000000).random_(1,6)
train_ds = TensorDataset(X,y)
train_dl = DataLoader(train_ds, batch_size = 1024, num_workers = 8)

pl.Trainer(gpus = 1, max_epochs = 10, progress_bar_refresh_rate = 50, logger = False)\
.fit(mdacf, train_dl)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name     | Type | Params
----------------------------------
0 | user_mda | mDA  | 115   
1 | item_mda | mDA  | 82    
----------------------------------
95        Trainable params
102       Non-trainable params
197       Total params
0.001     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 977/977 [00:06<00:00, 140.45it/s, loss=11, train_loss_step=11.90, train_loss_epoch=10.90]  


1

In [6]:
for name, param in user_mda.named_parameters():
    print(name, param)
print("---")
for name, param in item_mda.named_parameters():
    print(name, param)
print("---")
for name, param in mdacf.named_parameters():
    print(name, param)

encoder.0.weight Parameter containing:
tensor([[-1.9957,  0.7173, -0.2455,  1.4919,  1.4798],
        [-0.7202, -0.6658, -1.6321, -1.8520,  0.1143],
        [-3.0071, -2.7380,  0.5236, -1.4199, -0.1604],
        [-2.7923, -2.3588, -2.6229, -2.8551,  0.4946],
        [-0.5634, -0.7447, -0.7998, -0.9337,  0.1127],
        [-1.7648, -1.7044, -1.7400, -0.5323, -0.0301],
        [ 1.1139,  0.0276, -0.1722, -0.3299,  1.0580],
        [-1.3731,  0.2638, -0.9001,  0.8823,  1.7379],
        [-1.4127,  1.6514,  0.6338, -0.2715,  0.6810],
        [ 1.5600, -1.0156, -2.3971, -1.8673,  0.8934]], requires_grad=True)
encoder.0.bias Parameter containing:
tensor([0.1632, 0.2446, 0.1561, 0.0780, 2.2812], requires_grad=True)
decoder.0.weight Parameter containing:
tensor([[-1.5801,  0.0823, -2.2548, -1.8616,  0.2046, -0.7802,  1.7302, -0.9573,
         -1.0228,  2.1623],
        [ 1.2416,  0.2807, -1.9206, -1.3982,  0.1066, -0.6785,  0.5265,  0.7678,
          2.1975, -0.5494],
        [ 0.2086, -0.8817, 